In [1]:
%reload_ext autoreload
%autoreload 2

# Load the data

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [3]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm
from time import sleep

DEVICE = "ibm_sherbrooke"
LOGICAL = str(1)
XBASIS = True
ROUNDS = 50

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
while True:
    try:
        md = metadata_loader(True, True)
        break
    except:
        sleep(5)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

md = md[0:1]
print("shape:", md.shape)
# md[18:]
md

state = X1
shape: (1, 18)


,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,num_qubits,job_status,execution_date,code,distance,rounds,logical,descr,resets,xbasis,path_info
5114,2024-03-23 14:14:43.354000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzdbgydvs8g008j69y0,"[Subset 52, 50 rounds, 1 log, xbasis=True]",1.0,1507.0,NaN,JobStatus.DONE,2024-03-23 15:55:36.002897+01:00,RepetitionCodeCircuit,52,50,1,subset RepCodes,NaN,True,"{'mean_gate_error': 0.007733045348037, 'min_ga..."


# Take single job

In [4]:
job_id = md["job_id"].values[0]
d = md["distance"].values[0]
T = md["rounds"].values[0]

job = provider.retrieve_job(job_id)
memory = job.result().get_memory()

print(f"nb of shots: {len(memory)}")

nb of shots: 1507


# Get needed decoding objects

In [5]:
from soft_info import load_calibration_memory
import cpp_soft_info as csi

lin = [0.6, 1.2, 1]
num_points = 20
bandwidths = np.linspace(lin[0], lin[1], lin[2])
rel_error = 1

all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_job=job_id, 
                                             double_msmt=False, post_process=True)
kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)


all_memories_PS, _, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_job=job_id, 
                                                         double_msmt=True, post_process=True)
kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)


Specified job execution date: 2024-03-23 15:55:36.002897+01:00
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 14:56:07.669856+00:00.
Specified job execution date: 2024-03-23 15:55:36.002897+01:00
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 14:56:37.347221+00:00.


In [6]:
print(all_memories[0].keys())
print(all_memories_PS[0].keys())

dict_keys(['mmr_0', 'mmr_1'])
dict_keys(['mmr_0', 'mmr_1', 'mmr_0_scnd', 'mmr_1_scnd'])


In [7]:
# Get the msmt errors
p_soft_mean = 0
p_hard_mean = 0
for key, value in msmt_err_dict_PS.items():
    p_soft_mean += value['p_soft']
    p_hard_mean += value['p_hard']
p_soft_mean /= len(msmt_err_dict_PS)
p_hard_mean /= len(msmt_err_dict_PS)

print(f"p_soft_mean = {p_soft_mean}")
print(f"p_hard_mean = {p_hard_mean}")

p_soft_mean = 0.008409516819033635
p_hard_mean = 0.021719793439586875


In [8]:
from soft_info import inv_qubit_mapping, get_repcode_IQ_map

layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
link_qubits = list(layout_des['link_qubit'].values())
code_qubits = list(layout_des['code_qubit'].values())

big_layout = link_qubits + code_qubits
print(big_layout)
inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))
print(inverted_q_map)

[23, 25, 27, 35, 48, 50, 36, 31, 17, 11, 9, 7, 5, 3, 1, 14, 19, 33, 40, 42, 44, 54, 65, 67, 69, 74, 88, 86, 84, 92, 103, 105, 107, 112, 125, 123, 121, 119, 110, 99, 91, 80, 72, 61, 59, 71, 76, 90, 95, 109, 115, 22, 24, 26, 28, 47, 49, 51, 32, 30, 12, 10, 8, 6, 4, 2, 0, 18, 20, 39, 41, 43, 45, 64, 66, 68, 70, 89, 87, 85, 83, 102, 104, 106, 108, 126, 124, 122, 120, 118, 100, 98, 79, 81, 62, 60, 58, 77, 75, 94, 96, 114, 116]
{23: [0, 51, 102, 153, 204, 255, 306, 357, 408, 459, 510, 561, 612, 663, 714, 765, 816, 867, 918, 969, 1020, 1071, 1122, 1173, 1224, 1275, 1326, 1377, 1428, 1479, 1530, 1581, 1632, 1683, 1734, 1785, 1836, 1887, 1938, 1989, 2040, 2091, 2142, 2193, 2244, 2295, 2346, 2397, 2448, 2499], 25: [1, 52, 103, 154, 205, 256, 307, 358, 409, 460, 511, 562, 613, 664, 715, 766, 817, 868, 919, 970, 1021, 1072, 1123, 1174, 1225, 1276, 1327, 1378, 1429, 1480, 1531, 1582, 1633, 1684, 1735, 1786, 1837, 1888, 1939, 1990, 2041, 2092, 2143, 2194, 2245, 2296, 2347, 2398, 2449, 2500], 27: [2,

# Get countMat and pSoft

In [9]:
from datetime import datetime
from soft_info import gaussianIQConvertor

HANDLE_OUTLIERS = True

print(f"Starting to get pSoft and countMat at {datetime.now()}")
pSoft, countMat = csi.iqConvertor(memory, inverted_q_map, kde_dict, rel_error, -1,
                                            handleOutliers = HANDLE_OUTLIERS)

print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
pSoft_PS, countMat_PS = csi.iqConvertor(memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                handleOutliers = HANDLE_OUTLIERS)

# print(f"Starting to get pSoftG at {datetime.now()}")
# countMatG, pSoftG = gaussianIQConvertor(memory, inverted_q_map, gmm_dict)

Starting to get pSoft and countMat at 2024-04-10 08:53:33.707732
Starting to get pSoft_PS and countMat_PS at 2024-04-10 08:54:05.458632


# Decode 1 subset, 1 distance

In [10]:
from soft_info import generate_subsets_with_center, get_cols_to_keep, get_subsample_layout

D_NEW = 11
subsets = generate_subsets_with_center(d, D_NEW)
subset = subsets[0]
cols = get_cols_to_keep(subset, T, d)
# cols = np.random.permutation(cols)
cols = [c+1 for c in cols] # because the first column is the syndrome

pSoft_sub = pSoft[:, cols]
pSoft_PS_sub = pSoft_PS[:, cols]
countMat_sub = countMat[:, cols]
countMat_PS_sub = countMat_PS[:,  cols]

In [11]:
print(f"shape pSoft: {pSoft.shape}")
print(f"subset: {subset}")
print(f"cols: {cols}")

shape pSoft: (1507, 2602)
subset: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
cols: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 970

# Get the noise model 

In [12]:
from soft_info import get_noise_dict_from_backend, get_avgs_from_dict

layout = get_subsample_layout(subset, link_qubits, code_qubits)


noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = md["execution_date"].values[0])
avgs = get_avgs_from_dict(noise_dict, layout)
noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
readout = avgs["readout"]
noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]

print(f"noise_list = {noise_list}")
print(f"readout = {readout}")

08:54:31 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
08:54:31 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
08:54:31 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
noise_list = [0.007633334953803748, 0.0003265154690155472, 0.0012358423164544369, 0.004311102659255508, 0.03012931025862051, 0.021719793439586875, 0.008409516819033635]
readout = 0.012728571428571433


In [13]:
# Stim codes
from soft_info import RepetitionCodeStimCircuit

subsampling = True
_RESETS = False

noise_list_new = noise_list.copy()
# noise_list_new = [err*3 for err in noise_list_new]
# noise_list_new[2] = 30e-2 # t1 error

code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list_new,
                                                subsampling=subsampling, no_fin_soft=True, layout=None,
                                                msmt_err_dict=None)
model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list_new,
                                                subsampling=subsampling, no_fin_soft=False, layout=None,
                                                msmt_err_dict=None)
model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()

# Decoding

In [14]:
_RESETS_new = False

offset = 0.01
outcome = 1
# pSoft_new = pSoft_sub + offset
# pSoft_PS_new = pSoft_PS_sub + offset

pSoft_new = pSoft_sub.copy()
pSoft_PS_new = pSoft_PS_sub.copy()
# biased offset
pSoft_new[countMat_sub == outcome] += offset
pSoft_PS_new[countMat_PS_sub == outcome] += offset
# set to 0.5
pSoft_new[pSoft_new > 0.5] = 0.5
pSoft_new[pSoft_new <= 0] = 1e-8
pSoft_PS_new[pSoft_PS_new > 0.5] = 0.5
pSoft_PS_new[pSoft_PS_new <= 0] = 1e-8


res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                      int(LOGICAL), _RESETS_new, decode_hard=False)
res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                      int(LOGICAL), _RESETS_new, decode_hard=True)

res_s_K_mean_PS = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                        int(LOGICAL), _RESETS_new, decode_hard=False)
res_h_K_mean_PS = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                        int(LOGICAL), _RESETS_new, decode_hard=True)

res_s_K_mean_new = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_new, T,
                                            int(LOGICAL), _RESETS_new, decode_hard=False)
res_s_K_mean_PS_new = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_new, T,
                                            int(LOGICAL), _RESETS_new, decode_hard=False)

In [15]:
res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_sub, T,
                                      int(LOGICAL), _RESETS_new, decode_hard=False)
res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_sub, T,
                                      int(LOGICAL), _RESETS_new, decode_hard=True)

res_s_K_mean_PS = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS_sub, pSoft_PS_sub, T,
                                        int(LOGICAL), _RESETS_new, decode_hard=False)
res_h_K_mean_PS = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS_sub, pSoft_PS_sub, T,
                                        int(LOGICAL), _RESETS_new, decode_hard=True)

res_s_K_mean_new = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_new, T,
                                            int(LOGICAL), _RESETS_new, decode_hard=False)
res_s_K_mean_PS_new = csi.decodeConvertorAll(model_mean_for_hard, countMat_sub, pSoft_new, T,
                                            int(LOGICAL), _RESETS_new, decode_hard=False)

In [16]:
print(f"D_NEW: {D_NEW}, T: {T}, SHOTS: {len(pSoft)}, LOGICAL: {LOGICAL}, XBASIS: {XBASIS}, RESETS: {_RESETS}, DOUBLE_MSMTS: {True}")
print(f"noise_list: {noise_list}")
print(f"offset {offset} on outcome {outcome}\n")

print(f"soft KDE: {res_s_K_mean.num_errors} out of {len(pSoft)} shots, handlWeOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")
print(f"soft KDE_new: {res_s_K_mean_new.num_errors} out of {len(pSoft)} shots, handlWeOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")
print(f"hard KDE: {res_h_K_mean.num_errors} out of {len(pSoft)} shots, handlWeOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")

print() 

print(f"soft KDE_PS: {res_s_K_mean_PS.num_errors} out of {len(pSoft_PS)} shots, handlWeOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")
print(f"soft KDE_new_PS: {res_s_K_mean_PS_new.num_errors} out of {len(pSoft_PS)} shots, handlWeOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")
print(f"hard KDE_PS: {res_h_K_mean_PS.num_errors} out of {len(pSoft_PS)} shots, handlWeOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")


D_NEW: 11, T: 50, SHOTS: 1507, LOGICAL: 1, XBASIS: True, RESETS: False, DOUBLE_MSMTS: True
noise_list: [0.007633334953803748, 0.0003265154690155472, 0.0012358423164544369, 0.004311102659255508, 0.03012931025862051, 0.021719793439586875, 0.008409516819033635]
offset 0.01 on outcome 1

soft KDE: 137 out of 1507 shots, handlWeOutliers: True, bandwidth: [0.6]
soft KDE_new: 133 out of 1507 shots, handlWeOutliers: True, bandwidth: [0.6]
hard KDE: 180 out of 1507 shots, handlWeOutliers: True, bandwidth: [0.6]

soft KDE_PS: 192 out of 1507 shots, handlWeOutliers: True, bandwidth: [0.6]
soft KDE_new_PS: 133 out of 1507 shots, handlWeOutliers: True, bandwidth: [0.6]
hard KDE_PS: 180 out of 1507 shots, handlWeOutliers: True, bandwidth: [0.6]


In [17]:
print(f"p_soft_mean: {p_soft_mean}, p_hard_mean: {p_hard_mean}")
print(f"mean pSoft: {np.mean(pSoft)}, mean pSoft_PS: {np.mean(pSoft_PS)}")
print(f"mean pSoft_new: {np.mean(pSoft_new)}, mean pSoft_PS_new: {np.mean(pSoft_PS_new)}")
print(f"mean pSoft_sub: {np.mean(pSoft_sub)}, mean pSoft_PS_sub: {np.mean(pSoft_PS_sub)}")

p_soft_mean: 0.008409516819033635, p_hard_mean: 0.021719793439586875
mean pSoft: 0.029747324059849346, mean pSoft_PS: 0.010276452570038673
mean pSoft_new: 0.026470830398796236, mean pSoft_PS_new: 0.013601674332388004
mean pSoft_sub: 0.022463922487557388, mean pSoft_PS_sub: 0.009590898728967777
